In [1]:
import scanpy as sc

In [2]:
import rapids_singlecell as rapids

In [3]:
import sklearn

In [4]:
sklearn.__version__

'1.4.2'

In [5]:
# ad_path_organoids = '/mnt/f/workspace/theislab/mubind/data/organoids_treutlein_dataset/RNA_ATAC_metacells_sce_peaks.h5ad'
ad_paths = ['/mnt/f/workspace/theislab/mubind/data/pancreatic_endocrinogenesis/pancreas_multiome_2022_processed_atac.h5ad',
            '/mnt/f/workspace/theislab/mubind/data/noack_2022/merged_scATAC_integrated_cicero_faye_chong.h5ad',
            '/mnt/f/workspace/theislab/mubind/data/organoids_treutlein_dataset/RNA_ATAC_metacells_sce_peaks.h5ad',
             '/mnt/f/workspace/theislab/mubind/data/pbmc/processed_laura_2023.h5ad',
             '/mnt/f/workspace/theislab/mubind/data/organoids_treutlein_dataset/RNA_ATAC_metacells_sce_peaks.h5ad']

In [6]:
ad_by_path = {}

In [7]:
ad_paths

['/mnt/f/workspace/theislab/mubind/data/pancreatic_endocrinogenesis/pancreas_multiome_2022_processed_atac.h5ad',
 '/mnt/f/workspace/theislab/mubind/data/noack_2022/merged_scATAC_integrated_cicero_faye_chong.h5ad',
 '/mnt/f/workspace/theislab/mubind/data/organoids_treutlein_dataset/RNA_ATAC_metacells_sce_peaks.h5ad',
 '/mnt/f/workspace/theislab/mubind/data/pbmc/processed_laura_2023.h5ad',
 '/mnt/f/workspace/theislab/mubind/data/organoids_treutlein_dataset/RNA_ATAC_metacells_sce_peaks.h5ad']

In [8]:
import pandas as pd
import os

# the ratio of peaks over cells e.g. 3 : 1
ratio_feature_over_cells = 3

overwrite = False
for ad_path in ad_paths:
    # if not 'laura' in ad_path:
    #     continue

    if not 'pbmc' in ad_path:
        continue

    for selection in ['random', 'episcanpy']:
        if selection == 'random':
            print('')
        print('')
        # print(selection, ad_path)
        outdir = os.path.join(os.path.dirname(ad_path), selection)
        # print(outdir)
        if not os.path.exists(outdir):
            os.makedirs(outdir)

        # last = False
        for n_sample_cells in [100,]: # 200, 300,]: # 500, 1000, 2000, 5000, 10000, 'ALL']:

            # if n_sample_cells == 'ALL':
            #     continue

            n_sample_peaks = int(n_sample_cells * ratio_feature_over_cells) if n_sample_cells is not 'ALL' else 'ALL'
            adata = None

            outname = os.path.basename(ad_path.replace('.h5ad', '_obs%s_var%s.h5ad' % (n_sample_cells, n_sample_peaks)))
            outpath = os.path.join(outdir, outname)

            print(os.path.exists(outpath), outpath)

            # continue

            if os.path.exists(outpath) and not overwrite:
                continue

            print('attempting to read')

            # continue
            # assert False
            ad = sc.read_h5ad(ad_path) if not ad_path in ad_by_path else ad_by_path[ad_path]

            # if n_sample_cells is not None:
            # if n_sample_cells < ad.shape[0]:
            print(n_sample_cells)
            adata = sc.pp.subsample(ad,
                                    n_obs=ad.shape[0] if isinstance(n_sample_cells, str) else n_sample_cells,
                                    copy=True,
                                    random_state=ad.shape[0] if isinstance(n_sample_cells, str) else n_sample_cells)
            # else:
            #     adata = ad
            #     last = True
            # else:
            #     adata = ad

            print('after cells selection', adata.shape)

            # sc.pp.highly_variable_genes(adata, n_top_genes=2500)
            # using episcanpy
            print('selection rule', selection)
            if selection == 'episcanpy':
                acc = (adata.X != 0).sum(axis=0) / adata.shape[0]
                acc_score = abs(.5 - acc)
                adata.var['acc_score'] = acc_score.A.T
                adata.var['acc_score_rank'] = adata.var['acc_score'].rank(ascending=False)
                # plt.hist(atac.var['acc_score'], bins=35)

                var_sample = adata.var.sort_values('acc_score_rank', ascending=False).index # [:n_sample_peaks if n_sample_peaks is not 'ALL' else adata.shape[1]]

                # select until getting N peaks with non-zero sums
                var_subset = []
                n_non_zero = 0
                for v, n in zip(var_sample, adata.X.sum(axis=0).A.flatten()):
                    var_subset.append(v)
                    if n != 0:
                        n_non_zero += 1
                    if n_non_zero >= (n_sample_peaks if n_sample_peaks is not 'ALL' else adata.shape[1]):
                        break
                adata = adata[:,adata.var_names.isin(set(var_subset))].copy()
                # adata = adata[:,adata.var_names.isin(var_sample)].copy()


            elif selection == 'random':
                # obs_sample = pd.Series(adata.obs_names).sample(n_sample_cells)
                var_sample = pd.Series(adata.var_names).sample(n_sample_peaks if isinstance(n_sample_peaks, int) else adata.shape[1])
                # var_sample = atac.var.sort_values('acc_score_rank').index[:n_sample_peaks]
                # adata = adata[adata.obs_names.isin(obs_sample),adata.var_names.isin(var_sample)].copy()
                adata = adata[:,adata.var_names.isin(var_sample)].copy()

            print('after peaks selection', adata.shape)

            print('sum of reads', adata.X.sum())
            if adata.X.sum() == 0:
                print('skip...not too many peaks after selection...')
                continue

            # adata.shape
            sc.pp.neighbors(adata) #  method='rapids')
            sc.tl.umap(adata, init_pos="random", random_state=1) #  method='rapids')

            # out of memory in very large cases
            # rapids.pp.neighbors(adata) #  method='rapids')
            # rapids.tl.umap(adata, init_pos="random", random_state=1) #  method='rapids')

            # adata.X.A.max()
            # sc.pl.umap(adata, color='celltype')
            assert outpath != ad_path

            # adata.shape
            adata.write(outpath, compression='lzf')

            # if last:
            #     break

<>:31: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:84: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:31: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:84: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/tmp/ipykernel_264460/1493364190.py:31: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  n_sample_peaks = int(n_sample_cells * ratio_feature_over_cells) if n_sample_cells is not 'ALL' else 'ALL'
/tmp/ipykernel_264460/1493364190.py:84: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if n_non_zero >= (n_sample_peaks if n_sample_peaks is not 'ALL' else adata.shape[1]):




False /mnt/f/workspace/theislab/mubind/data/pbmc/random/processed_laura_2023_obs100_var300.h5ad
attempting to read
100
after cells selection (100, 165434)
selection rule random
after peaks selection (100, 300)
sum of reads 2890.0
         Falling back to preprocessing with `sc.pp.pca` and default params.


/home/ilibarra/.conda/envs/mubind/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



False /mnt/f/workspace/theislab/mubind/data/pbmc/episcanpy/processed_laura_2023_obs100_var300.h5ad
attempting to read
100
after cells selection (100, 165434)
selection rule episcanpy
after peaks selection (100, 360)
sum of reads 27540.0
         Falling back to preprocessing with `sc.pp.pca` and default params.
